In [1]:
import numpy as np
import sympy
import sympy as sy
from sympy import symbols, factorial, Function, summation, binomial, product
# import scipy.optimize

n, i, j, m = symbols('n, i, j, m', integer=True, real=True)

t = symbols('t')
P = Function('P') #Position Function as a function of time

deg = 6 #Bezier degree

B = summation(binomial(n, i)*(1 - t)**(n-i)*t**i*P(i), (i, 0, n))
P_vect = (sympy.Matrix([P(i) for i in range(deg)])).T #Bezier 6 uses range(6)?
P_vect

Matrix([[P(0), P(1), P(2), P(3), P(4), P(5)]])

In [2]:
class Bezier:
#https://en.wikipedia.org/wiki/B%C3%A9zier_curve

    def __init__(self, P, T: float):
        self.P = P
        self.m = P.shape[0]
        self.n = P.shape[1]-1
        self.T = T
    
    def eval(self, t):
        #https://en.wikipedia.org/wiki/De_Casteljau%27s_algorithm
        beta = t/self.T
        A = sympy.Matrix(self.P)
        for j in range(1, self.n + 1):
            for k in range(self.n + 1 - j):
                A[:, k] = A[:, k] * (1 - beta) + A[:, k + 1] * beta
        return A[:, 0]

    def deriv(self, m=1):
        D_sy = sympy.Matrix(self.P)
        diff_calc = sympy.zeros(D_sy.shape[0]-1,1).T
        for j in range(1, m+1): #should range start from 1 or 0???
            diff_calc = diff_calc.row_join(difference_operator(D_sy,j))
            D_soln = (P_vect.shape[1] -j)*diff_calc  #constant term is 5? or 4? P_vect.shape[1]-1?
        return Bezier(D_soln/self.T**m, self.T)
    
def difference_operator(P_vect, j=1): #check if j must be 1
    return sympy.Matrix([P_vect[i+1] - P_vect[i] for i in range(len(P_vect) - j)]).T
    
def derive_bezier6():
    n = 6 #Bezier Degree
    T = sympy.Symbol('T') #ca.SX.sym('T')
    t = sympy.Symbol('t') #ca.SX.sym('t')                       
    P= Function('P')       # P = ca.SX.sym('P', 1, n)
    P_c = (sympy.Matrix([P(i) for i in range(n)])).T #Bezier Positon Matrices length = n
    B = Bezier(P_c, T)

    # derivatives
    B_d = B.deriv()
    B_d2 = B_d.deriv()
    B_d3 = B_d2.deriv()
    B_d4 = B_d3.deriv()

    # boundary conditions

    # trajectory
    p = B.eval(t)
    v = B_d.eval(t)
    a = B_d2.eval(t)
    r = sy.Matrix.vstack(p,v,a)    #ca.vertcat(p, v, a)

    # given position/velocity boundary conditions, solve for bezier points
    wp_0 = sympy.MatrixSymbol('ptin', 2, 1) #ca.SX.sym('p0', 2, 1)  # pos/vel at waypoint 0
    wp_1 = sympy.MatrixSymbol('ptone', 2, 1) #ca.SX.sym('p1', 2, 1)  # pos/vel at waypoint 1

    constraints = []
    constraints += [(B.eval(0), wp_0[0])]  # pos @ wp0
    constraints += [(B_d.eval(0), wp_0[1])]  # vel @ wp0
    constraints += [(B_d2.eval(0), 0)]  # zero accel @ wp0
    constraints += [(B.eval(T), wp_1[0])]  # pos @ wp1
    constraints += [(B_d.eval(T), wp_1[1])]  # vel @ wp1
    constraints += [(B_d2.eval(T), 0)]  # zero accel @ wp1

    assert len(constraints) == 6

    Y = sy.Matrix.vstack(*[c[0] for c in constraints])  #ca.vertcat(*[c[0] for c in constraints])
    b = sy.Matrix([c[1] for c in constraints])   #ca.vertcat(*[c[1] for c in constraints])
    A = Y.jacobian(P_c)
    A_inv = A.inv()
    P_sol = (A_inv@b).T
    return {
        'bezier6_solve': P_sol
    }



In [3]:
#RUNNING BEZIER constraint coefficient matrix

wp_0 = sympy.MatrixSymbol('ptin', 2, 1) #ca.SX.sym('p0', 2, 1)  # pos/vel at waypoint 0
wp_1 = sympy.MatrixSymbol('ptone',2,1)
T = sympy.Symbol('T')

#INPUTS
wpVal0 = [0,1] #Waypoint value in [pos, vel] for wp0
wpVal1 = [2,3] #Waypoint value in [pos, vel] for wp1
T_l = 4 #input Time T

p_constr = derive_bezier6()['bezier6_solve'] #Bezier Coefficient
for i in range(wp_0.shape[0]):
    p_constr = p_constr.subs(wp_0[i],wpVal0[i]).subs(wp_1[i],wpVal1[i])
    
# p_constr = p_constr.subs(T,T_l) #Substitute T value

p_constr

Matrix([[0, T/5, 2*T/5, 2 - 6*T/5, 2 - 3*T/5, 2]])

In [4]:
# n0 =6
# C = sympy.factorial(n)/sympy.factorial(n - j)*summation((-1)**(i + j)*P(i)/(factorial(i)*factorial(j - i)), (i, 0, j)) ##error with sympy version when run

# P_vect = sympy.Matrix([P(i) for i in range(n0)])
# C_matrix = sympy.Matrix([C.subs({j: j0, n: n0}).doit() for j0 in range(n0)])
# C_matrix

# A = C_matrix.jacobian(P_vect)